In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [46]:
!pwd
%cd ./drive/My Drive/Colab Notebooks/UniverseDacon/
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
sample_submission.describe()

/content/drive/My Drive/Colab Notebooks/UniverseDacon
[Errno 2] No such file or directory: './drive/My Drive/Colab Notebooks/UniverseDacon/'
/content/drive/My Drive/Colab Notebooks/UniverseDacon


,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
count,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# TRAIN 데이터의 TYPE을 SAMPLE_SUBMISSION에 대응하는 가변수 형태로 변환시킵니다.
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

train 에서 type column 을 없애고 test 랑 합쳐서 .

IQR 방법으로 Outlier 제거.
일단 주석처리.

In [50]:
y_train = train['type_num']
train_wo_type = train.drop(columns=['type','type_num','fiberID'], axis=1)
test = test.drop(columns=['fiberID'])
total = pd.concat([train_wo_type, test])
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210000 entries, 0 to 10008
Data columns (total 21 columns):
id            210000 non-null int64
psfMag_u      210000 non-null float64
psfMag_g      210000 non-null float64
psfMag_r      210000 non-null float64
psfMag_i      210000 non-null float64
psfMag_z      210000 non-null float64
fiberMag_u    210000 non-null float64
fiberMag_g    210000 non-null float64
fiberMag_r    210000 non-null float64
fiberMag_i    210000 non-null float64
fiberMag_z    210000 non-null float64
petroMag_u    210000 non-null float64
petroMag_g    210000 non-null float64
petroMag_r    210000 non-null float64
petroMag_i    210000 non-null float64
petroMag_z    210000 non-null float64
modelMag_u    210000 non-null float64
modelMag_g    210000 non-null float64
modelMag_r    210000 non-null float64
modelMag_i    210000 non-null float64
modelMag_z    210000 non-null float64
dtypes: float64(20), int64(1)
memory usage: 35.2 MB


In [0]:
# from collections import Counter
# def detect_outliers(df, n, features): 
#   outlier_indices = [] 
#   for col in features:
#     if col == 'type' or col == 'id':
#       continue; 
#     Q1 = np.percentile(df[col], 25) 
#     Q3 = np.percentile(df[col], 75) 
#     IQR = Q3 - Q1 

#     outlier_step =  IQR 
#     outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index 
#     outlier_indices.extend(outlier_list_col) 

#   outlier_indices = Counter(outlier_indices) 
#   multiple_outliers = list(k for k, v in outlier_indices.items() if v > n) 
#   print()
#   return multiple_outliers 

# features = total.columns
# print(features)
# dup = detect_outliers(total, 2, features)

# print(len(dup))

# filter(lambda a: a < 199991, dup)
# print(len(dup))

In [51]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import minmax_scale
#Scaler = StandardScaler()
#Scaler = MinMaxScaler()
Scaler = RobustScaler()

total[['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i','fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i','petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i','modelMag_z']] = Scaler.fit_transform(total[['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i','psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i','fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i','petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i','modelMag_z']])
total.describe()
X_train = total.iloc[:199991]
test = total.iloc[199991:]
X_train.describe()


,id,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199991.00000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000
mean,99995.00000,-1.101418e+01,-0.501477,-0.426805,-0.429742,-0.385359,-4.254289,-0.401149,-0.197641,-0.414012,-0.280835,0.576861,-0.391497,-0.235831,-0.338996,-0.158222,-0.108654,-0.347379,-0.322532,-0.317283,-0.292707
std,57732.57318,4.736054e+03,63.422130,51.541291,50.121313,50.339846,1740.348334,301.626939,35.820649,50.408619,57.739039,309.613080,53.381038,32.669793,49.084836,47.514679,44.853596,56.003328,44.905767,43.853669,42.936817
min,0.00000,-2.117774e+06,-16339.744058,-11029.366423,-11425.722863,-10129.148855,-777873.716137,-86915.744311,-8680.696066,-8331.910727,-10499.545334,-9601.988638,-8982.926052,-8052.432652,-13626.619779,-10019.242823,-9629.838573,-12785.353886,-12219.397613,-13033.426412,-8372.746487
25%,49997.50000,-4.856811e-01,-0.490946,-0.569784,-0.556883,-0.482187,-0.462277,-0.469522,-0.545760,-0.529228,-0.437089,-0.438920,-0.509134,-0.572382,-0.553322,-0.456080,-0.418478,-0.509526,-0.576621,-0.556210,-0.447999
50%,99995.00000,3.220121e-04,-0.000025,0.000215,0.000205,0.000432,0.000201,-0.000052,0.000035,0.000132,0.000249,-0.000053,0.000078,-0.000228,0.000097,0.000076,-0.000150,0.000044,-0.000242,0.000081,0.000117
75%,149992.50000,5.143601e-01,0.508444,0.430554,0.442716,0.517910,0.537660,0.530047,0.454508,0.470798,0.563226,0.561008,0.490346,0.428033,0.446631,0.543865,0.581542,0.489932,0.423726,0.443718,0.552081
max,199990.00000,7.478071e+03,1436.000816,1227.902537,2069.872112,3989.069635,2022.746137,99860.116617,4799.382370,3311.908758,7259.900950,117163.206091,4190.996475,2346.392975,3285.157201,5789.200524,5306.131089,3657.592355,4095.212771,1351.832486,2377.503900


In [52]:
test.describe()

,id,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000,10009.000000
mean,204995.000000,0.046595,-0.010552,-0.070441,-0.073170,0.003111,0.056686,0.008941,-0.053144,-0.043297,0.046032,0.136652,-0.042985,-0.063319,-0.025788,0.079129,0.121671,-0.004399,-0.069974,-0.039849,0.057952
std,2889.493756,0.842075,1.050305,0.693016,0.739520,0.692529,0.830497,0.750816,0.680249,0.673035,0.677605,1.101012,4.831036,0.664959,0.663688,0.790188,0.802099,0.677846,0.622257,0.600664,0.599828
min,199991.000000,-11.212789,-25.531652,-4.183713,-17.863895,-2.140323,-4.863068,-4.782681,-2.921033,-2.677166,-10.753661,-46.492203,-472.914949,-14.477725,-9.110849,-27.668522,-2.929437,-2.053433,-1.930852,-1.900922,-1.832304
25%,202493.000000,-0.484511,-0.503251,-0.574132,-0.559310,-0.482521,-0.462266,-0.473873,-0.548037,-0.528554,-0.435257,-0.438278,-0.512477,-0.573966,-0.555571,-0.455796,-0.417651,-0.513507,-0.576296,-0.558136,-0.444816
50%,204995.000000,-0.006439,0.002463,-0.003663,-0.004355,-0.006520,-0.003286,0.001003,-0.001363,-0.003229,-0.004163,0.001580,-0.001168,0.004570,-0.002969,-0.000799,0.002000,-0.002177,0.004009,-0.002112,-0.001525
75%,207497.000000,0.514507,0.508339,0.420003,0.447136,0.515832,0.538358,0.536781,0.447505,0.470975,0.557905,0.563449,0.497718,0.418218,0.445482,0.546617,0.581568,0.496514,0.417383,0.444323,0.553477
max,209999.000000,6.703486,66.412049,5.039392,12.112770,6.645779,8.393391,10.906150,3.832852,4.925694,3.075022,17.657821,30.213413,7.615469,11.292996,18.838138,4.486858,3.209131,2.827309,2.617526,2.043127


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, \
                                                  test_size=0.2, random_state=0)

In [0]:
%%time
from xgboost import XGBClassifier

XGB_clf = XGBClassifier(n_esetimators=1000, n_jobs=-1, random_state=0)
XGB_clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)])

[0]	validation_0-merror:0.250788	validation_1-merror:0.252106
[1]	validation_0-merror:0.229668	validation_1-merror:0.232531
[2]	validation_0-merror:0.226855	validation_1-merror:0.228481
[3]	validation_0-merror:0.224555	validation_1-merror:0.226781
[4]	validation_0-merror:0.223592	validation_1-merror:0.226181
[5]	validation_0-merror:0.224155	validation_1-merror:0.226331
[6]	validation_0-merror:0.222967	validation_1-merror:0.225106
[7]	validation_0-merror:0.222142	validation_1-merror:0.223581
[8]	validation_0-merror:0.218617	validation_1-merror:0.220156
[9]	validation_0-merror:0.217036	validation_1-merror:0.21913
[10]	validation_0-merror:0.215905	validation_1-merror:0.218405
[11]	validation_0-merror:0.215242	validation_1-merror:0.217405
[12]	validation_0-merror:0.215048	validation_1-merror:0.21738
[13]	validation_0-merror:0.212461	validation_1-merror:0.214705
[14]	validation_0-merror:0.211742	validation_1-merror:0.21443
[15]	validation_0-merror:0.211836	validation_1-merror:0.214205
[16]	

In [0]:
XGB_pred = XGB_clf.predict_proba(test)